<a href="https://colab.research.google.com/github/StefanG994/LLM-fine-tuning/blob/main/Fine_tune_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Update pip and install necessary libraries
!pip install --upgrade pip
!pip install xformers --no-build-isolation
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install transformers datasets accelerate trl unsloth

Looking in indexes: https://download.pytorch.org/whl/cu121


In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"Is CUDA available? {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.5.1+cu121
Is CUDA available? True
GPU: Tesla T4


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving enhanced_fine_tune_data_flatten.json to enhanced_fine_tune_data_flatten.json


In [ ]:
# Step 1: Imports and Setup
import json
import torch
from datasets import Dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel

# Load your JSON dataset
with open("enhanced_fine_tune_data_flatten.json", "r") as file:
    data = json.load(file)

# Convert JSON to Hugging Face Dataset
dataset = Dataset.from_list(data)

# Split into train and validation datasets
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

# Inspect a sample
print(train_dataset[0])


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
{'instruction': "Was sind die Indikationen für das Nahrungsergänzungsmittel 'Bio-Leinöl'?", 'response': 'Indikationen: Zur ausreichenden Versorgung mit mehrfach ungesättigten Fettsäuren, zur Vorbeugung arteriosklerotischer Erkrankungen, begleitend bei Darmerkrankungen, Durchfallerkran-kungen, Leaky-Gut-Syndrom, Neurodermitis sowie Ekzemerkrankungen. Zur Unterstüt-zung bei rheumatischen Erkrankungen, Arthritis, Polyneuropathie.'}


In [ ]:
# Step 4: Tokenize the Dataset
model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
# model_name = "unsloth/unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.11.11: Fast Mistral patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 42,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.11 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
# Define the prompt template
prompt = "Instruction: {instruction}\nResponse: {response}"

# Add the EOS_TOKEN
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    responses    = examples["response"]
    texts = []
    for instruction, response in zip(instructions, responses):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction=instruction, response=response) + EOS_TOKEN
        texts.append(text)
    return { "text": texts }

# Apply the formatting function to the datasets
train_dataset = train_dataset.map(formatting_prompts_func, batched=True, remove_columns=["instruction", "response"])
eval_dataset = eval_dataset.map(formatting_prompts_func, batched=True, remove_columns=["instruction", "response"])

Map:   0%|          | 0/3555 [00:00<?, ? examples/s]

Map:   0%|          | 0/395 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,  # Use the processed train_dataset
    eval_dataset = eval_dataset,    # Use the processed eval_dataset (optional)
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,  # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 160,
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",  # Use this for WandB etc.
    ),
)

Map (num_proc=2):   0%|          | 0/3555 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/395 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
4.637 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,555 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 160
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.434500
2,2.778500
3,2.854900
4,2.338700
5,2.410400
6,2.074000
7,2.334700
8,2.130700
9,1.806900
10,2.103400


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1079.5967 seconds used for training.
17.99 minutes used for training.
Peak reserved memory = 10.436 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 70.762 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [ ]:
#@title Inference
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt.format(
        instruction="Welche Nahrungsergänzungsmittel werden für folgende Indikationen empfohlen:  Zur Gefäßtherapie bei Arteriosklerose, Bluthochdruck, endotheliale Dysfunktion. Gefäßrisiko ist auch gleichzeitig Infarkt-, Apoplexie-, Hochdruck- und Nierenrisiko?",
        response="")
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Instruction: Welche Nahrungsergänzungsmittel werden für folgende Indikationen empfohlen:  Zur Gefäßtherapie bei Arteriosklerose, Bluthochdruck, endotheliale Dysfunktion. Gefäßrisiko ist auch gleichzeitig Infarkt-, Apoplexie-, Hochdruck- und Nierenrisiko?\nResponse:  Empfohlene Nahrungsergänzungsmittel: Boswellia Plus<|end_of_text|>']